In [2]:
import pandas as pd
import re
import json
import ast
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from Levenshtein import distance as lev
import time


In [3]:
food_path = "1M_recipes_with_nutritional_info.json"
flavor_path = "flavor_DB.json"

In [104]:
def flatten(path, cols, meta):
   # load data using Python JSON module
    with open(path,'r') as f:
        data = json.loads(f.read())
        
    # Flatten data
    df = pd.json_normalize(
        data, 
        record_path =cols, 
        meta=meta
    )
    return df


In [105]:
food_df = pd.read_json(food_path)
food_df.iloc[0:5, :]

,fsa_lights_per100g,id,ingredients,instructions,nutr_per_ingredient,nutr_values_per100g,partition,quantity,title,unit,url,weight_per_ingr
0,"{'fat': 'green', 'salt': 'green', 'saturates':...",000095fc1d,"[{'text': 'yogurt, greek, plain, nonfat'}, {'t...",[{'text': 'Layer all ingredients in a serving ...,"[{'fat': 0.8845044000000001, 'nrg': 133.80964,...","{'energy': 81.12946131894766, 'fat': 2.1401392...",train,"[{'text': '8'}, {'text': '1'}, {'text': '1/4'}]",Yogurt Parfaits,"[{'text': 'ounce'}, {'text': 'cup'}, {'text': ...",http://tastykitchen.com/recipes/breakfastbrunc...,"[226.796, 152.0, 30.5]"
1,"{'fat': 'red', 'salt': 'orange', 'saturates': ...",00051d5b9d,"[{'text': 'sugars, granulated'}, {'text': 'oil...",[{'text': 'Cream sugar and butter together til...,"[{'fat': 0.0, 'nrg': 384.0, 'pro': 0.0, 'sat':...","{'energy': 477.09640393594606, 'fat': 23.41248...",val,"[{'text': '1/2'}, {'text': '3/4'}, {'text': '1...","Salt Free, Low Cholesterol Sugar Cookies Recipe","[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://cookeatshare.com/recipes/salt-free-low-...,"[100.80000000000001, 168.0, 21.25, 5.166666666..."
2,"{'fat': 'orange', 'salt': 'orange', 'saturates...",00059b093b,"[{'text': 'chicken, broilers or fryers, wing, ...","[{'text': 'Preheat oven to 400 degrees F.'}, {...","[{'fat': 174.859716, 'nrg': 2599.0821599999995...","{'energy': 208.05898280960727, 'fat': 14.29704...",train,"[{'text': '3'}, {'text': '1'}, {'text': '1/2'}...",Honey Sriracha Chicken Wings,"[{'text': 'pound'}, {'text': 'teaspoon'}, {'te...",http://tastykitchen.com/recipes/main-courses/h...,"[1360.7759999999998, 6.0, 1.15, 13.5, 42.59999..."
3,"{'fat': 'orange', 'salt': 'orange', 'saturates...",0005fc89f7,"[{'text': 'crustaceans, shrimp, raw (not previ...","[{'text': 'In a large bowl, toss the shrimp, g...","[{'fat': 4.6266384, 'nrg': 771.1064, 'pro': 18...","{'energy': 194.7525956849092, 'fat': 15.980767...",train,"[{'text': '2'}, {'text': '3/4'}, {'text': '1 1...",Shrimp and Caper Salad,"[{'text': 'pound'}, {'text': 'cup'}, {'text': ...",http://allrecipes.com/recipe/shrimp-and-caper-...,"[907.184, 75.0, 151.5, 25.799999999999997, 134..."
4,"{'fat': 'orange', 'salt': 'green', 'saturates'...",0006ca31f4,"[{'text': 'cocoa, dry powder, unsweetened'}, {...",[{'text': 'Measure out the cocoa powder into a...,"[{'fat': 141.35999999999999, 'nrg': 2352.0, 'p...","{'energy': 303.43539955190437, 'fat': 5.094846...",train,"[{'text': '12'}, {'text': '12'}, {'text': '1'}]",Natural Peanut Butter Chocolate Bon Bons,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://www.food.com/recipe/natural-peanut-butt...,"[1032.0, 4068.0, 256.0]"


In [106]:
## read file
flavor_df = pd.read_json(flavor_path);
flavor_df.iloc[:5, :]

,category,entity_id,category_readable,entity_alias_basket,entity_alias_readable,natural_source_name,entity_alias,molecules,natural_source_url,entity_alias_url,entity_alias_synonyms
0,animalproduct,0,Animal Product,"egg, egg-boiled, egg-cooked, egg-scrambled",Egg,Chicken,egg,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Chicken,https://en.wikipedia.org/wiki/Egg_as_food,Egg
1,bakery,1,Bakery,bakery-products,Bakery Products,Poacceae,bakery,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Poaceae,https://en.wikipedia.org/wiki/Bakery,Bakery Products
2,bakery,2,Bakery,"bread, bread-preferment",Bread,Poacceae,bread,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Poaceae,https://en.wikipedia.org/wiki/Bread,Bread
3,bakery,3,Bakery,bread-rye,Rye Bread,Rye,bread-rye,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Rye,https://en.wikipedia.org/wiki/Rye_bread,Rye Bread
4,bakery,4,Bakery,bread-wheaten,Wheaten Bread,Wheat,bread-wheaten,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Wheat,https://en.wikipedia.org/wiki/Soda_bread,"Soda scones, Soda farls"


In [107]:
## flatten molecules from nested array
flatten_flavor_df = flatten(flavor_path, ["molecules"], ["entity_id", "category", "entity_alias_readable"])
flatten_flavor_df.iloc[0:3,:]

,bond_stereo_count,undefined_atom_stereocenter_count,taste,functional_groups,fooddb_flavor_profile,supersweetdb_id,fema_number,fooddb_id,common_name,hba_count,...,hbd_count,fema_flavor_profile,complexity,heavy_atom_count,defined_atom_stereocenter_count,monoisotopic_mass,atom_stereo_count,entity_id,category,entity_alias_readable
0,0,0,,amine@primary amine@primary aliphatic amine (a...,faint,,3819,FDB002257,L-arginine,4,...,4,,176.0,12,1,174.112,1,0,animalproduct,Egg
1,0,0,"warm, over-ripe, fruity flavor below 0.1 ppm",aromatic compound@heterocyclic compound,very strong@mothball@animal@fecal@indole@civet,,3019,FDB010993,3-Methylindole,0,...,1,,122.0,10,0,131.073,0,0,animalproduct,Egg
2,0,0,,aromatic compound@heterocyclic compound,roast beef@roasted nuts@grass@medical@cocoa@wo...,,3272,FDB013954,"2,5-Dimethylpyrazine",2,...,0,"Cocoa, Roast Beef, Roasted Nut",62.9,8,0,108.069,0,0,animalproduct,Egg


In [108]:
## flatten ingredients from nested array
flatten_food_df = flatten(food_path, ["ingredients"], ["id", "title"])
flatten_food_df = flatten_food_df.iloc[0:200, :] # partition to a small subset for testing

In [109]:
## re-order the column header and rename ingredient
flatten_food_df = flatten_food_df[["id", "title", "text"]]
flatten_food_df = flatten_food_df.rename(columns = {"text": "ingredient"})
flatten_food_df.iloc[0:10, :]

,id,title,ingredient
0,000095fc1d,Yogurt Parfaits,"yogurt, greek, plain, nonfat"
1,000095fc1d,Yogurt Parfaits,"strawberries, raw"
2,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade"
3,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","sugars, granulated"
4,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","oil, corn, peanut, and olive"
5,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","egg substitute, powder"
6,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","orange juice, raw"
7,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","orange juice, raw"
8,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","leavening agents, baking powder, double-acting..."
9,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","wheat flour, white, all-purpose, unenriched"


In [110]:
flatten_flavor_df = flatten_flavor_df[["category", "entity_alias_readable", "fooddb_id", "pubchem_id", "common_name", "iupac_name"]]
flatten_flavor_df.iloc[0:10, :]

,category,entity_alias_readable,fooddb_id,pubchem_id,common_name,iupac_name
0,animalproduct,Egg,FDB002257,6322,L-arginine,(2S)-2-amino-5-(diaminomethylideneamino)pentan...
1,animalproduct,Egg,FDB010993,6736,3-Methylindole,3-methyl-1H-indole
2,animalproduct,Egg,FDB013954,31252,"2,5-Dimethylpyrazine","2,5-dimethylpyrazine"
3,animalproduct,Egg,FDB008174,7909,4-Methyl-2-pentanone,4-methylpentan-2-one
4,animalproduct,Egg,FDB008127,7284,2-Methylbutyraldehyde,2-methylbutanal
5,animalproduct,Egg,FDB012552,7501,Styrene,styrene
6,animalproduct,Egg,FDB008326,9609,Diethyl sulfide,ethylsulfanylethane
7,animalproduct,Egg,FDB012084,12180,Methyl butyrate,methyl butanoate
8,animalproduct,Egg,FDB003349,18827,1-Octen-3-Ol,oct-1-en-3-ol
9,animalproduct,Egg,FDB008541,18635,3-(Methylthio)propionaldehyde,3-methylsulfanylpropanal


In [111]:
# ## fuzzywuzzy implementation
# ingredients = flavor_df["entity_alias_readable"].unique()
# start = time.process_time()
# food_df['key']=food_df['ingredient'].apply(lambda x : [process.extract(x, ingredients, limit=1)][0][0][0])
# duration = time.process_time() - start 
# print(duration)    

In [112]:
## levenshtein implementation
ingredients = flatten_flavor_df["entity_alias_readable"].unique()
start = time.process_time()
flatten_food_df['key']=flatten_food_df['ingredient'].apply(lambda x : min(ingredients, key=lambda i : lev(x, i))) 
duration = time.process_time() - start 
print(duration)  
flatten_food_df.iloc[0:10, :]

0.19167099999999948


,id,title,ingredient,key
0,000095fc1d,Yogurt Parfaits,"yogurt, greek, plain, nonfat",Northern bluefin tuna
1,000095fc1d,Yogurt Parfaits,"strawberries, raw",Strawberry Jam
2,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",Wholewheat Bread
3,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","sugars, granulated",Pomegranate
4,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","oil, corn, peanut, and olive",European anchovy
5,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","egg substitute, powder",Sugar substitute
6,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","orange juice, raw",Orange Oil
7,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","orange juice, raw",Orange Oil
8,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","leavening agents, baking powder, double-acting...",Mandarin Orange Peel Oil
9,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","wheat flour, white, all-purpose, unenriched",Breadnut tree seed


In [113]:
merged_df = flatten_flavor_df.merge(flatten_food_df,left_on='entity_alias_readable',right_on='key', how="inner")
merged_df.iloc[[0,3], :]

,category,entity_alias_readable,fooddb_id,pubchem_id,common_name,iupac_name,id,title,ingredient,key
0,bakery,Wheaten Bread,FDB011267,263034,Difurfuryl ether,2-(furan-2-ylmethoxymethyl)furan,0007c8edef,Easy Chocolate Frosting Recipe,"cheese, parmesan, hard",Wheaten Bread
3,bakery,Wheaten Bread,FDB011267,263034,Difurfuryl ether,2-(furan-2-ylmethoxymethyl)furan,00180ff35c,Potato and Hamburger Baskets #SP5,"cheese, cheddar",Wheaten Bread


In [114]:
## reorder the cols
merged_df = merged_df[["id", "title", "ingredient", "category", "entity_alias_readable", "fooddb_id", "pubchem_id", "common_name", "iupac_name"]]
merged_df = merged_df.rename(columns = {"title": "food", "entity_alias_readable": "ingredient_flavorDB"})
merged_df = merged_df.sort_values(by=["id"])

In [115]:
## flatten col "nutrition per 100g"
## then join
# with open(food_path,'r') as f:
#     data = json.loads(f.read())
# nutr = pd.json_normalize(data)
# flatten_nutr = nutr.filter(regex="nutr_values_per100g")
# flatten_nutr.insert(0, "title", nutr["title"], True)
# flatten_nutr.insert(0, "id", nutr["id"], True)

In [116]:
merged_df.info()
merged_df.iloc[0:5, :]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9736 entries, 2328 to 4188
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   9736 non-null   object
 1   food                 9736 non-null   object
 2   ingredient           9736 non-null   object
 3   category             9736 non-null   object
 4   ingredient_flavorDB  9736 non-null   object
 5   fooddb_id            9736 non-null   object
 6   pubchem_id           9736 non-null   int64 
 7   common_name          9736 non-null   object
 8   iupac_name           9736 non-null   object
dtypes: int64(1), object(8)
memory usage: 760.6+ KB


,id,food,ingredient,category,ingredient_flavorDB,fooddb_id,pubchem_id,common_name,iupac_name
2328,000095fc1d,Yogurt Parfaits,"strawberries, raw",fruit-berry,Strawberry Jam,FDB011707,22873,Hexyl Hexanoate,hexyl hexanoate
204,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,,61209,"3-Methylcyclopentane-1,2-dione","3-methylcyclopentane-1,2-dione"
205,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB003296,11579,Levulinic Acid,4-oxopentanoic acid
206,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB012137,8194,Dodecanal,dodecanal
207,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB003311,5283335,trans-2-Nonenal,(E)-non-2-enal


In [137]:
## make set of molecules for each food
unique_molecules = merged_df[["id", "food", "ingredient_flavorDB", "fooddb_id", "pubchem_id", "common_name"]]
unique_molecules = unique_molecules.drop_duplicates(subset=["id", "pubchem_id"])
unique_molecules

,id,food,ingredient_flavorDB,fooddb_id,pubchem_id,common_name
2328,000095fc1d,Yogurt Parfaits,Strawberry Jam,FDB011707,22873,Hexyl Hexanoate
204,000095fc1d,Yogurt Parfaits,Wholewheat Bread,,61209,"3-Methylcyclopentane-1,2-dione"
205,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB003296,11579,Levulinic Acid
206,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB012137,8194,Dodecanal
207,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB003311,5283335,trans-2-Nonenal
...,...,...,...,...,...,...
5605,00234c37b5,Cherry Tomato Corn Salad,Pomegranate,FDB013255,5280445,luteolin
4769,00234c37b5,Cherry Tomato Corn Salad,Pomegranate,FDB003303,31289,Nonanal
1315,00234c37b5,Cherry Tomato Corn Salad,Lime Peel Oil,FDB014467,61362,d-Piperitone
5616,00234c37b5,Cherry Tomato Corn Salad,Pomegranate,FDB002594,637542,p-coumaric acid


In [138]:
unique_molecules.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5424 entries, 2328 to 4188
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   5424 non-null   object
 1   food                 5424 non-null   object
 2   ingredient_flavorDB  5424 non-null   object
 3   fooddb_id            5424 non-null   object
 4   pubchem_id           5424 non-null   int64 
 5   common_name          5424 non-null   object
dtypes: int64(1), object(5)
memory usage: 296.6+ KB


In [139]:
## make tuples of molecule attributes
columns = ["ingredient_flavorDB", "fooddb_id", "pubchem_id", "common_name"]
unique_molecules["molecules"] = unique_molecules[columns].to_dict(orient="records")
unique_molecules = unique_molecules[["id", "food", "molecules"]]
unique_molecules = unique_molecules.groupby(["id", "food"], as_index=False).agg(flavor_molecules=("molecules", lambda x : list(x)))
unique_molecules

,id,food,flavor_molecules
0,000095fc1d,Yogurt Parfaits,"[{'ingredient_flavorDB': 'Strawberry Jam', 'fo..."
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[{'ingredient_flavorDB': 'Pomegranate', 'foodd..."
2,00059b093b,Honey Sriracha Chicken Wings,"[{'ingredient_flavorDB': 'Honey', 'fooddb_id':..."
3,0005fc89f7,Shrimp and Caper Salad,"[{'ingredient_flavorDB': 'Celery', 'fooddb_id'..."
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,"[{'ingredient_flavorDB': 'Cocoa powder', 'food..."
5,00073a6b36,Easy Cheese Sauce,[{'ingredient_flavorDB': 'Spineless Monkey Ora...
6,0007c8edef,Easy Chocolate Frosting Recipe,"[{'ingredient_flavorDB': 'Pomegranate', 'foodd..."
7,0007f5f800,Cornmeal Crackers with Pumpkin Seeds,"[{'ingredient_flavorDB': 'Butternut squash', '..."
8,0008c6e39a,Chocolate-Orange Angel Food Cake,"[{'ingredient_flavorDB': 'Pomegranate', 'foodd..."
9,0009785a2e,"Rice With Lemon, Capers & Parsley","[{'ingredient_flavorDB': 'Butternut squash', '..."


In [140]:
unique_molecules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                32 non-null     object
 1   food              32 non-null     object
 2   flavor_molecules  32 non-null     object
dtypes: object(3)
memory usage: 896.0+ bytes


In [141]:
## flatten flavor molecules into list for each food
unique_molecules["molecules_count"] = unique_molecules.apply(lambda x : len(x['flavor_molecules']), axis=1)
unique_molecules.iloc[0:5, :]
unique_molecules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                32 non-null     object
 1   food              32 non-null     object
 2   flavor_molecules  32 non-null     object
 3   molecules_count   32 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.1+ KB


In [146]:
## add attributes: fsa_lights_per100g, ingredients, nutr_values_per100g

final_df = unique_molecules.merge(food_df[["id","instructions", "ingredients", "nutr_values_per100g", "fsa_lights_per100g"]], on="id", how="inner")
# reorder column
final_df = final_df[["id", "food", "flavor_molecules", "ingredients", "molecules_count", "instructions", "nutr_values_per100g", "fsa_lights_per100g"]]

final_df


,id,food,flavor_molecules,ingredients,molecules_count,instructions,nutr_values_per100g,fsa_lights_per100g
0,000095fc1d,Yogurt Parfaits,"[{'ingredient_flavorDB': 'Strawberry Jam', 'fo...","[{'text': 'yogurt, greek, plain, nonfat'}, {'t...",83,[{'text': 'Layer all ingredients in a serving ...,"{'energy': 81.12946131894766, 'fat': 2.1401392...","{'fat': 'green', 'salt': 'green', 'saturates':..."
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[{'ingredient_flavorDB': 'Pomegranate', 'foodd...","[{'text': 'sugars, granulated'}, {'text': 'oil...",172,[{'text': 'Cream sugar and butter together til...,"{'energy': 477.09640393594606, 'fat': 23.41248...","{'fat': 'red', 'salt': 'orange', 'saturates': ..."
2,00059b093b,Honey Sriracha Chicken Wings,"[{'ingredient_flavorDB': 'Honey', 'fooddb_id':...","[{'text': 'chicken, broilers or fryers, wing, ...",267,"[{'text': 'Preheat oven to 400 degrees F.'}, {...","{'energy': 208.05898280960727, 'fat': 14.29704...","{'fat': 'orange', 'salt': 'orange', 'saturates..."
3,0005fc89f7,Shrimp and Caper Salad,"[{'ingredient_flavorDB': 'Celery', 'fooddb_id'...","[{'text': 'crustaceans, shrimp, raw (not previ...",259,"[{'text': 'In a large bowl, toss the shrimp, g...","{'energy': 194.7525956849092, 'fat': 15.980767...","{'fat': 'orange', 'salt': 'orange', 'saturates..."
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,"[{'ingredient_flavorDB': 'Cocoa powder', 'food...","[{'text': 'cocoa, dry powder, unsweetened'}, {...",75,[{'text': 'Measure out the cocoa powder into a...,"{'energy': 303.43539955190437, 'fat': 5.094846...","{'fat': 'orange', 'salt': 'green', 'saturates'..."
5,00073a6b36,Easy Cheese Sauce,[{'ingredient_flavorDB': 'Spineless Monkey Ora...,"[{'text': 'cheese, pasteurized process, americ...",33,"[{'text': 'Put milk, onion salt and pepper sau...","{'energy': 57.123531501115025, 'fat': 2.477641...","{'fat': 'green', 'salt': 'red', 'saturates': '..."
6,0007c8edef,Easy Chocolate Frosting Recipe,"[{'ingredient_flavorDB': 'Pomegranate', 'foodd...","[{'text': 'spartan, real semi-sweet chocolate ...",168,[{'text': 'Heat chocolate on top of double boi...,"{'energy': 367.7230503667077, 'fat': 4.3421179...","{'fat': 'orange', 'salt': 'orange', 'saturates..."
7,0007f5f800,Cornmeal Crackers with Pumpkin Seeds,"[{'ingredient_flavorDB': 'Butternut squash', '...","[{'text': 'cornmeal, degermed, unenriched, yel...",156,"[{'text': 'In a large skillet, stir the cornme...","{'energy': 399.5182190718795, 'fat': 18.337440...","{'fat': 'orange', 'salt': 'red', 'saturates': ..."
8,0008c6e39a,Chocolate-Orange Angel Food Cake,"[{'ingredient_flavorDB': 'Pomegranate', 'foodd...","[{'text': 'sugars, powdered'}, {'text': 'wheat...",164,[{'text': 'Move oven rack to lowest position.'...,"{'energy': 149.42599870037444, 'fat': 0.367917...","{'fat': 'green', 'salt': 'green', 'saturates':..."
9,0009785a2e,"Rice With Lemon, Capers & Parsley","[{'ingredient_flavorDB': 'Butternut squash', '...","[{'text': 'rice, white, long-grain, regular, u...",281,[{'text': 'Add rice to a large pan of boiling ...,"{'energy': 57.34966592427616, 'fat': 3.4322726...","{'fat': 'orange', 'salt': 'red', 'saturates': ..."


In [147]:
final_df["flavor_molecules"].iloc[1]

[{'ingredient_flavorDB': 'Pomegranate',
  'fooddb_id': 'FDB014627',
  'pubchem_id': 6986,
  'common_name': '2-Isopropyl-5-methylcyclohexanone'},
 {'ingredient_flavorDB': 'Orange Oil',
  'fooddb_id': 'FDB003994',
  'pubchem_id': 111037,
  'common_name': 'Alpha-Terpinyl Acetate'},
 {'ingredient_flavorDB': 'Pomegranate',
  'fooddb_id': 'FDB013710',
  'pubchem_id': 441005,
  'common_name': '(+)-delta-Cadinene'},
 {'ingredient_flavorDB': 'Orange Oil',
  'fooddb_id': 'FDB011944',
  'pubchem_id': 1146,
  'common_name': 'trimethylamine'},
 {'ingredient_flavorDB': 'Orange Oil',
  'fooddb_id': 'FDB003013',
  'pubchem_id': 5283361,
  'common_name': 'trans-2-Dodecenal'},
 {'ingredient_flavorDB': 'Breadnut tree seed',
  'fooddb_id': 'FDB014891',
  'pubchem_id': 445070,
  'common_name': 'farnesol'},
 {'ingredient_flavorDB': 'Pomegranate',
  'fooddb_id': 'FDB008307',
  'pubchem_id': 7847,
  'common_name': 'Acrolein'},
 {'ingredient_flavorDB': 'Pomegranate',
  'fooddb_id': 'FDB001112',
  'pubchem_id':

In [148]:
## write into json file
final_df.iloc[0:1, :].to_json(r'res.json', orient="records")

In [104]:
## find all unique molecules
m_unique = set()
for i in range(len(flatten_molecule["flavor_molecules"])):
    for j in flatten_molecule["flavor_molecules"].iloc[i]:
        if len(j) > 0:
            m_unique.add(j)
len(m_unique)

623

In [21]:
test = final_df.ilocl